In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [17]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [18]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

25 documents
5 classes ['goodbye', 'greeting', 'house', 'motivate', 'thanks']
72 unique lemmatized words ["'s", ',', '.', 'a', 'am', 'anyone', 'are', 'awesome', 'because', 'bye', 'chatting', 'day', 'depressed', 'difficult', 'dont', 'feel', 'for', 'good', 'goodbye', 'great', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'household', 'how', 'i', 'in', 'is', 'it', 'just', 'kind', 'later', 'like', 'lot', 'm', 'many', 'me', 'motivated', 'my', 'myself', 'next', 'nice', 'not', 'noticed', 'of', 'off', 'or', 'place', 'putting', 's', 'see', 'seems', 'some', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'too', 'traped', 'when', 'work', 'world', 'you', '’']


In [19]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [20]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
72/72 [==============================] - 1s 20ms/step - loss: 0.3496 - acc: 0.9861
Epoch 2/200
72/72 [==============================] - 0s 3ms/step - loss: 0.0088 - acc: 1.0000
Epoch 3/200
72/72 [==============================] - 0s 3ms/step - loss: 0.0019 - acc: 1.0000
Epoch 4/200
72/72 [==============================] - 0s 4ms/step - loss: 0.0102 - acc: 1.0000
Epoch 5/200
72/72 [==============================] - 0s 4ms/step - loss: 0.0044 - acc: 1.0000
Epoch 6/200
72/72 [==============================] - 0s 4ms/step - loss: 0.0046 - acc: 1.0000
Epoch 7/200
72/72 [==============================] - 0s 4ms/step - loss: 0.0017 - acc: 1.0000
Epoch 8/200
72/72 [==============================] - 0s 3ms/step - loss: 0.0054 - acc: 1.0000
Epoch 9/200
72/72 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 10/200
72/72 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 11/200
72/72 [==============================] - 0s 2

72/72 [==============================] - 0s 3ms/step - loss: 1.1520e-05 - acc: 1.0000
Epoch 84/200
72/72 [==============================] - 0s 3ms/step - loss: 3.7973e-04 - acc: 1.0000
Epoch 85/200
72/72 [==============================] - 0s 4ms/step - loss: 4.3908e-04 - acc: 1.0000
Epoch 86/200
72/72 [==============================] - 0s 3ms/step - loss: 3.3949e-04 - acc: 1.0000
Epoch 87/200
72/72 [==============================] - 0s 4ms/step - loss: 3.0226e-04 - acc: 1.0000
Epoch 88/200
72/72 [==============================] - 0s 4ms/step - loss: 4.0943e-04 - acc: 1.0000
Epoch 89/200
72/72 [==============================] - 0s 4ms/step - loss: 7.2648e-06 - acc: 1.0000
Epoch 90/200
72/72 [==============================] - 0s 4ms/step - loss: 1.1882e-04 - acc: 1.0000
Epoch 91/200
72/72 [==============================] - 0s 4ms/step - loss: 2.0015e-04 - acc: 1.0000
Epoch 92/200
72/72 [==============================] - 0s 4ms/step - loss: 6.7175e-04 - acc: 1.0000
Epoch 93/200
72/72 [===

72/72 [==============================] - 0s 2ms/step - loss: 2.4099e-06 - acc: 1.0000
Epoch 164/200
72/72 [==============================] - 0s 2ms/step - loss: 2.2297e-05 - acc: 1.0000
Epoch 165/200
72/72 [==============================] - 0s 2ms/step - loss: 1.5189e-04 - acc: 1.0000
Epoch 166/200
72/72 [==============================] - 0s 2ms/step - loss: 9.2821e-06 - acc: 1.0000
Epoch 167/200
72/72 [==============================] - 0s 2ms/step - loss: 3.8591e-04 - acc: 1.0000
Epoch 168/200
72/72 [==============================] - 0s 2ms/step - loss: 2.0214e-04 - acc: 1.0000
Epoch 169/200
72/72 [==============================] - 0s 2ms/step - loss: 1.3425e-05 - acc: 1.0000
Epoch 170/200
72/72 [==============================] - 0s 2ms/step - loss: 8.5418e-05 - acc: 1.0000
Epoch 171/200
72/72 [==============================] - 0s 2ms/step - loss: 4.7081e-06 - acc: 1.0000
Epoch 172/200
72/72 [==============================] - 0s 2ms/step - loss: 1.3087e-05 - acc: 1.0000
Epoch 173/200


In [21]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents) 
    return res

In [22]:
chatbot_response("I am too trapped in my household work")

'If you follow our online material you can easily manage house hold work along with learning through the turtorials and it will eventually help you to become less dependant on others'

In [25]:
chatbot_response("I dont want to do anythanything")

Every person is special, you can do anything that you want


In [ ]:
chatbot_reponse("")